<a href="https://colab.research.google.com/github/sahandv/science_science/blob/master/FastText_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FASTTEXT EMBEDDING


## Init

### Clone Project Git Repo

In [1]:
!rm -rf 'science_science'
username = "sahandv"#@param {type:"string"}
# password = ""#@param {type:"string"} 

!git clone https://github.com/$username/science_science.git
!ls

Cloning into 'science_science'...
remote: Enumerating objects: 66, done.
remote: Counting objects: 100% (66/66), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 344 (delta 34), reused 17 (delta 0), pack-reused 278
Receiving objects: 100% (344/344), 70.41 MiB | 11.40 MiB/s, done.
Resolving deltas: 100% (143/143), done.
sample_data  science_science


### Mount Google Drive


In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [3]:
# Check files!
!ls 'drive/My Drive/Data-Permenant/FastText-crawl-300d-2M-subword'

crawl-300d-2M-subword.bin  crawl-300d-2M-subword.vec


### Install requirements

In [4]:
!pip install fasttext
!pip install -r 'science_science/requirements.txt'

     |████████████████████████████████| 61kB 1.9MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.1-cp36-cp36m-linux_x86_64.whl size=2386106 sha256=47b55958af7c389218d37b4f919051f23d4f5ae133c79e0b856cb1376104b6d9
  Stored in directory: /root/.cache/pip/wheels/9f/f0/04/caa82c912aee89ce76358ff954f3f0729b7577c8ff23a292e3
Successfully built fasttext
     |████████████████████████████████| 1.6MB 2.7MB/s 
     |████████████████████████████████| 61kB 7.2MB/s 
     |████████████████████████████████| 552kB 45.7MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97711 sha256=33fa60a1482f957baddc1562d6b74df5b890a6a56e3edaa077b5893e37576833
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
  Created wheel for funcy: filename=funcy-1.14-py2.py3-none-any.whl size=32042 sha256=4b1a74002e8307ab554ab481580ca5c0326e41c5f08ee923e8af360acba59d52
  Stored in directory: /root/.cache/pip/wheels/20/5a/d8/1d875df

### Import Libs

In [0]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import gc
import json
import re

import fasttext
from gensim.models import FastText as fasttext_gensim
from gensim.test.utils import get_tmpfile

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from science_science.sciosci.assets import keyword_assets as kw
from science_science.sciosci.assets import generic_assets as sci
from science_science.sciosci.assets import advanced_assets as aa


In [0]:
import warnings
warnings.filterwarnings("ignore")

In [6]:
stops = ['a','an','we','result','however','yet','since','previously','although','propose','proposed','e_g','method',
         'published_elsevier','b','v','problem','paper','approach','within','with','by','via','way','t','case','issue','level','area','system',
         'work','discussed','seen','put','usually','take','make','author','versus','enables','result','research','design','based']
punkts = [' ','','(',')','[',']','{','}','.',',','!','?','<','>','-','_',':',';','\\','/','|','&','%',"'s","`s",'#','$','@']

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

stop_words = list(set(stopwords.words("english")))+stops+punkts
np.random.seed(50)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


# Get embeddings from a pre-trained model


### Load Corpus

In [0]:
period = '1990-2018'
percentile = 97

#### Option A - Load corpus

In [0]:
directory = 'drive/My Drive/Data/corpus/improved_copyr_lemmatized_stopwords_removed_thesaurus_n-grams/'
file_name = period+' corpus abstract-title - with n-grams'
corpus = pd.read_csv(directory+file_name,names=['abstracts'])
corpus_tokens = [item for sublist in corpus['abstracts'].values.tolist() for item in sublist.split()]
gc.collect()

99

#### Option B - Load keywords

In [0]:
directory = 'drive/My Drive/Data/LDA/'
file_name = period+' top_90-percentile_keywords_terms.csv'
corpus = pd.read_csv(directory+file_name)
corpus = corpus.fillna('this_is_null')
corpus_tokens = []
for idx,row in tqdm(corpus.iterrows(),total=corpus.shape[0]):
    for token in row.values.tolist():
        if token != 'this_is_null': 
            corpus_tokens.append(token) 
del corpus
print("\nNumber of unique tokens:",len(corpus_tokens))

100%|██████████| 6546/6546 [00:00<00:00, 8418.10it/s]


Number of unique tokens: 52365


#### Option C - Load author keywords

In [0]:
directory = 'drive/My Drive/Data/Author keywords - 29 Oct 2019/'
file_name = period+' keyword frequency'
corpus = pd.read_csv(directory+file_name,names=['keyword','frequency'])
corpus = corpus.fillna('this_is_null')
threshold = np.percentile(corpus['frequency'].values,percentile)
corpus = corpus[corpus['frequency']>threshold]

corpus_tokens = []
for idx,row in tqdm(corpus.iterrows(),total=corpus.shape[0]):
    if row['keyword'] != 'this_is_null': 
        corpus_tokens.append(row['keyword']) 
print("\nNumber of unique tokens:",len(corpus_tokens))

100%|██████████| 1376/1376 [00:00<00:00, 9769.37it/s]


Number of unique tokens: 1376


## Facebook Model

#### Load model

In [0]:
fb_model_address = 'drive/My Drive/Data-Permenant/FastText-crawl-300d-2M-subword/crawl-300d-2M-subword.bin'

In [0]:
model = fasttext.load_model(fb_model_address)

#### Get embeddings

*   Save to dictionary and json
*   This takes much less space on disk


##### No n-gram handle

In [0]:
# Save in a dict
output_dict = {}
comment_embedding = ''
for token in tqdm(corpus_tokens[:],total=len(corpus_tokens[:])):
    output_dict[token] = str(model.get_word_vector(token))

100%|██████████| 280/280 [00:00<00:00, 337.81it/s]


##### Manual n-gram handle

In [0]:
# Save in a dict
comment_embedding = 'average_manual '
output_dict = {}
for token in tqdm(corpus_tokens[:],total=len(corpus_tokens[:])):
    token_split = token.split(' ')
    if len(token_split) > 0:
        tmp_vector_grams = []
        for item in token_split:
            tmp_vector_grams.append(model.get_word_vector(item))
        output_dict[token] = str(list(np.array(tmp_vector_grams).mean(axis=0)))
    else:
        output_dict[token] = str(model.get_word_vector(item))

100%|██████████| 280/280 [00:00<00:00, 3792.22it/s]


##### Save to disk

In [0]:
# Save embeddings to disk
with open(directory+'vectors/100D/FastText vector '+comment_embedding+period+'.json', 'w') as json_file:
    json.dump(output_dict, json_file)

#### Get embeddings (alternative) : 

*   save to a list instead of a dicktionary and csv
*   Will have many redundant words in it and will take lots of disk space




In [0]:
# Save in a list
batches = 1000
batch_size = len(corpus_tokens)/batches

for step in tqdm(range(batches),total=batches):
    batch_tokens = corpus_tokens[int(step*batch_size):int((step+1)*batch_size)]
    corpus_vectors = [model.get_word_vector(x) for x in batch_tokens]
    corpus_vectors = pd.DataFrame(corpus_vectors)
    corpus_vectors['tokens'] = batch_tokens

    # Save embeddings to disk
    with open(directory+'vector '+period,'a') as f:
        corpus_vectors.to_csv(f,index=False,header=False)

## Gensim Model

#### Load model

Load gensim model

In [0]:
gensim_model_address = 'drive/My Drive/Data/FastText Models/50D/fasttext-scopus_wos-merged-310k_docs-gensim 50D.model'
model = fasttext_gensim.load(gensim_model_address)

Simple Tests

In [0]:
print('intelligence' in model.wv.vocab)
print(model.similarity("anns", "ann"))
print(model.most_similar(positive=['baghdad', 'england'], negative=['london']))
print(model.n_similarity(['neural network','deep learning'], ['ann']))
print(model.wmdistance(['stop', 'word', 'removed', 'tokens', 'of', 'sentence 1'], ['stop word removed tokens of sentence 2']))

True
0.899301
[('bagua', 0.7456873059272766), ('baselevel', 0.7439467310905457), ('bagel', 0.7432736754417419), ('category', 0.7406319975852966), ('nine-category', 0.7370117902755737), ('baget', 0.734655499458313), ('categorized-and', 0.7324073910713196), ('categorizer', 0.7313241958618164), ('category.9', 0.730506420135498), ('eight-category', 0.7295982837677002)]
0.7487254
21.418109944602243


In [0]:
# get distance of two words
from scipy import spatial,sparse,sign
vec_a = model.wv['']
vec_b = model.wv['fpga']
distance_tmp = spatial.distance.cosine(vec_a, vec_b)
similarity_tmp = 1 - distance_tmp
similarity_tmp

0.33358505368232727

In [0]:
model.wv['artificial intelligence']

array([-4.25525   ,  0.03749213,  1.8276842 , -3.1528432 , -3.3440664 ,
        0.66207427,  1.0964861 , -2.038055  ,  3.0331683 , -2.1755521 ,
        2.1063838 ,  1.6578307 ,  1.3311137 , -2.030598  , -0.69794494,
        2.6208954 ,  1.9154872 ,  1.6715113 ,  0.23561044, -0.50721526,
        3.1775064 , -2.069317  , -2.4310536 , -1.8514946 ,  1.3029549 ,
        3.482592  , -2.1535952 ,  1.078043  , -3.8000522 ,  0.08382007,
       -0.6016187 ,  3.3550935 ,  2.5037699 , -2.8812122 , -0.11693893,
       -0.51311666,  3.1224    ,  0.46978405, -0.4427654 , -2.5400903 ,
        2.0880878 ,  3.123557  ,  0.8703581 , -1.0431769 , -2.8512125 ,
        2.2627175 ,  1.0080537 ,  0.1098367 ,  1.5881126 , -1.870272  ],
      dtype=float32)

In [0]:
(model.wv['artificial']+model.wv['intelligence'])/2

In [0]:
model.wv['artificial_intelligence']

### Get Word Embeddings

##### No n-gram handle

In [0]:
# Save in a dict
comment_embedding = ''
output_dict = {}
for token in tqdm(corpus_tokens[:],total=len(corpus_tokens[:])):
    output_dict[token] = str(model.wv[token])

100%|██████████| 3184174/3184174 [27:52<00:00, 1903.51it/s]


##### Manual n-gram handle

In [0]:
# Save in a dict
comment_embedding = 'average_manual '
output_dict = {}
for token in tqdm(corpus_tokens[:],total=len(corpus_tokens[:])):
    token_split = token.split(' ')
    if len(token_split) > 0:
        tmp_vector_grams = []
        for item in token_split:
            tmp_vector_grams.append(model.wv[item])
        output_dict[token] = str(list(np.array(tmp_vector_grams).mean(axis=0)))
    else:
        output_dict[token] = str(model.wv[token])

100%|██████████| 1590/1590 [00:00<00:00, 10574.09it/s]


##### Save to disk

In [0]:
# Save embeddings to disk
with open(directory+'/FastText vector with n-grams '+comment_embedding+period+'.json', 'w') as json_file:
    json.dump(output_dict, json_file)

In [0]:
print(directory)

drive/My Drive/Data/corpus/improved_copyr_lemmatized_stopwords_removed_thesaurus_n-grams/


### Get Doc Embeddings (SIF)

It is not recommended to perform this on cloud, as it is not process intesive, yet takes too long depending on the doc-count. It might take over 30 hours.

Make a probability dictionary

In [0]:
corpus_tokens_s = pd.Series(corpus_tokens)
corpus_tokens_probabilities = (corpus_tokens_s.groupby(corpus_tokens_s).transform('count') / len(corpus_tokens_s)).values
corpus_tokens_probabilities = pd.DataFrame(corpus_tokens_probabilities)
corpus_tokens_probabilities['tokens'] = corpus_tokens_s
corpus_tokens_probabilities.columns = ['probability','token']
corpus_tokens_probabilities = corpus_tokens_probabilities.groupby('token').mean()
corpus_tokens_probabilities = corpus_tokens_probabilities.reset_index()
corpus_tokens_probabilities.columns = ['token','probability']

Get vectors

In [0]:
vectors = []
for token in tqdm(corpus_tokens_probabilities['token'],total=corpus_tokens_probabilities.shape[0]):
    vectors.append(model.wv[token])
corpus_tokens_probabilities['vector'] = vectors

100%|██████████| 65533/65533 [00:03<00:00, 20827.02it/s]


Calculate weighted average vectors

In [0]:
a = 1e-3
embedding_size = 50

doc_set = []
for doc in tqdm(corpus['abstracts'].values.tolist(),total=len(corpus['abstracts'].values.tolist())):
    vs = np.zeros(embedding_size)  # add all word2vec values into one vector for the sentence
    doc_length = len(doc.split())
    print(doc.split())
    for word in doc.split():
        a_value = a / (a + corpus_tokens_probabilities[corpus_tokens_probabilities['token']==word]['probability'].values.tolist()[0])  # smooth inverse frequency, SIF
        vs = np.add(vs, np.multiply(a_value, corpus_tokens_probabilities[corpus_tokens_probabilities['token']==word]['vector'].values.tolist()[0]))  # vs += sif * word_vector

    vs = np.divide(vs, doc_length)  # weighted average
    doc_set.append(vs)  # add to our existing re-calculated set of sentences


# Train on a large Scopus corpus


#### Load Corpus Sentences

In [0]:
sentence_corpus = pd.read_csv('drive/My Drive/Data/corpus/AI ALL/1900-2019 corpus sentences abstract-title')

#### Preprocess and prepare corpus for FastText training

In [9]:
sentences = []
lemmatizer=WordNetLemmatizer()
with open('drive/My Drive/Data/corpus/AI ALL/1900-2019 corpus sentences abstract-title further processed.csv', 'w') as f:
    for index,row in tqdm(sentence_corpus.iterrows(),total=sentence_corpus.shape[0]):
        sentence = row['sentence']
        sentence = re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",sentence)
        sentence = word_tokenize(sentence)
        sentence = [word for word in sentence if not word in punkts] 
        sentence = [lemmatizer.lemmatize(word) for word in sentence]
        # sentences.append(sentence)
        f.write("%s\n" % ' '.join(sentence))

100%|██████████| 13307816/13307816 [1:54:08<00:00, 1943.10it/s]


#### Save sentences to disk for future use -- Not needed anymore

In [0]:
pd.DataFrame([' '.join(words) for words in sentences],columns=['sentences']).to_csv(
    'drive/My Drive/Data/corpus/AI ALL/1900-2019 corpus sentences abstract-title further processed.csv',
    header=True,index=False)

#### Load pre-processed sentences

In [10]:
sentence_corpus = pd.read_csv('drive/My Drive/Data/corpus/AI ALL/1900-2019 corpus sentences abstract-title further processed.csv',delimiter=";;;")
sentence_corpus.columns = ["sentences"]
sentences = []
sentence_corpus = sentence_corpus.fillna('')
for index,row in tqdm(sentence_corpus.iterrows(),total=sentence_corpus.shape[0]):
    sentences.append(row['sentences'].split(' '))

100%|██████████| 13230009/13230009 [28:30<00:00, 7736.00it/s]


In [11]:
sentence_corpus.head(10)

,sentences
0,circuit sequential circuit or vlsi chip realiz...
1,the design of efficient hardware is a fundamen...
2,because of the large cost for the physical con...
3,for these purpose data structure for boolean f...
4,the corresponding state of the art data struct...
5,efficient algorithm for the operation on obdds...
6,a generalized data structure called graph-driv...
7,the new data structure allows for many importa...
8,efficient algorithm for the operation on graph...
9,consider a finite graph g v e


### Train Fasttext - Gensim

#### Load a model to continue training

* If want to continue training, run this section

In [0]:
model = load(gensim_model_address)

In [0]:
model.build_vocab(sentences, update=True)
model.train(sentences, total_examples=len(sentences), epochs=model.epochs)

* Otherwise run this section

#### Train

In [0]:
model = fasttext_gensim(min_n=3, max_n=6, size=15, window=5, min_count=1, seed = 50)
model.build_vocab(sentences=sentences)
model.train(sentences=sentences, total_examples=len(sentences), epochs=10)

In [0]:
fname = "drive/My Drive/Data/Models/fasttext-scopus-2.2-million_docs-gensim 15D.model"
model.save(fname)

In [0]:
model = fasttext_gensim(min_n=3, max_n=6, size=50, window=5, min_count=1, seed = 50)
model.build_vocab(sentences=sentences)
model.train(sentences=sentences, total_examples=len(sentences), epochs=10)
fname = "drive/My Drive/Data/Models/fasttext-scopus-2.2-million_docs-gensim 50D.model"
model.save(fname)

In [0]:
model = fasttext_gensim(min_n=3, max_n=6, size=100, window=5, min_count=1, seed = 50)
model.build_vocab(sentences=sentences)
model.train(sentences=sentences, total_examples=len(sentences), epochs=10)
fname = "drive/My Drive/Data/Models/fasttext-scopus-2.2-million_docs-gensim 50D.model"
model.save(fname)

#### Test model

In [0]:
similarities = model.wv.most_similar(positive=['logic','fuzzy','expert'],negative=['deep','neural','network','cnn','ann'])
most_similar = similarities[0]

In [0]:
most_similar

('mam-rnn', 0.9763791561126709)

In [0]:
not_matching = model.wv.doesnt_match("human computer interface tree".split())

In [0]:
not_matching

'tree'

In [0]:
sim_score = model.wv.similarity('computer', 'human')

In [0]:
sim_score

0.7571839

In [0]:
print(model.wv['artificial intelligence'])
print(model.wv['artificial'])
print(model.wv['intelligence'])

[ 1.2783480e+00 -4.2018552e+00  7.1276689e-01  4.2023015e+00
 -5.0359420e-03  4.4385982e+00  6.2421050e+00 -8.9032326e+00
  1.7556003e+00  1.3425230e+00  9.4295764e-01 -4.4485557e-01
 -5.8648558e+00  2.6428668e+00 -1.2076639e+00]
[  3.7072854   -3.616749     1.3040072    0.234361    -2.753659
   7.528801    14.293305   -14.688236     5.3885765    6.496681
   1.9917868    2.855616    -0.05153261   7.8660994   -2.22459   ]
[  0.28606984  -6.971052    -0.9232919   11.48035      0.2561571
   4.084776     2.4220266   -8.616226     0.94255084  -2.2498865
   1.7112938   -3.370861   -12.577294    -1.1608386   -0.04991044]


### Train Fasttext - Facebook

In [0]:
sentences_joined = ' '.join(sentences)
model = fasttext.train_unsupervised(sentences_joined, "cbow", minn=2, maxn=5, dim=50, epoch=10,lr=0.05)

#### Test model

In [0]:
model.words

In [0]:
model.get_word_vector("the")

In [0]:
model.get_nearest_neighbors('asparagus')

In [0]:
model.get_analogies("intelligence", "math", "fuzzy")

#### Save model

In [0]:
model.save_model("drive/My Drive/Data/fasttext-scopus_wos-merged-310k_docs-facebook.ftz")